# Introdução ao Amazon SageMaker

Agora que concluímos todas as configurações necessárias, estamos prontos para treinar nosso detector de objetos. Para começar, vamos criar um objeto `` sageMaker.estimator.Estimator``. Este estimador iniciará o job de treinamento.

## Importando dependências

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import json
import numpy as np

role = get_execution_role()
sess = sagemaker.Session()
bucket= 'sagemaker-workshop-files-us-east-1-206730628141' # troque pelo seu bucket (o bucket foi criado pelo CloudFormation, verifique a aba de output no CloudFormation)
prefix = 'introduction'

In [ ]:
# Imagem pronta com o algoritmo built-in Image Classification
training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)

## Treinando o modelo

### Parâmetros para o treinamento
Existem dois tipos de parâmetros que precisam ser definidos para o treinamento. O primeiro são os parâmetros para o trabalho de treinamento. Esses incluem:

* Contagem de instâncias de treinamento: este é o número de instâncias nas quais o treinamento deve ser executado. Quando o número de instâncias for maior que um, o algoritmo de classificação de imagem será executado em configurações distribuídas.
* Tipo de instância de treinamento: indica o tipo de máquina na qual o treinamento será executado. Normalmente, usamos instâncias de GPU para treinamentos com imagem
* Caminho de saída: local no S3 onde a saída do treinamento é armazenada

In [ ]:
s3_output_location = f"s3://{bucket}/{prefix}/output"
ic = sagemaker.estimator.Estimator(
    training_image,
    role,
    train_instance_count=1,
    train_instance_type="ml.p2.xlarge",
    train_volume_size=50,
    train_max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

Além do conjunto de parâmetros acima, existem hiperparâmetros que são específicos do algoritmo. Esses são:

* **num_layers**: O número de camadas (profundidade) da rede. Usamos 18 neste exemplo, mas outros valores como 50, 152 podem ser usados.
* **image_shape**: As dimensões da imagem de entrada, 'num_channels, height, width', para a rede. Não deve ser maior do que o tamanho real da imagem. O número de canais deve ser igual à imagem real.
* **num_classes**: Este é o número de classes de saída para o novo conjunto de dados. Para caltech, usamos 257 porque tem 256 categorias de objetos + 1 classe de desordem.
* **num_training_samples**: Este é o número total de amostras de treinamento. Ele é definido como 15.240 para o conjunto de dados Caltech com a divisão atual.
* **mini_batch_size**: O número de amostras de treinamento usadas para cada minilote. No treinamento distribuído, o número de amostras de treinamento usadas por lote será N * mini_batch_size, onde N é o número de hosts nos quais o treinamento é executado.
* **epochs**: Número de epochs de treinamento.
* **learning_rate**: Taxa de aprendizagem para treinamento.
* **top_k**: Relata a precisão máxima durante o treinamento.
* **precision_dtype**: Precisão do tipo de dados de treinamento (padrão: float32). Se definido como 'float16', o treinamento será feito no modo mixed_precision e será mais rápido que o modo float32

In [ ]:
ic.set_hyperparameters(
    num_layers=20,
    image_shape="3,224,224",
    num_classes=257,
    num_training_samples=15420,
    mini_batch_size=128,
    epochs=20,
    learning_rate=0.01,
    top_k=2,
    precision_dtype="float32",
)

### Especificação dos dados de entrada

In [ ]:
s3train = f's3://{bucket}/{prefix}/train/'
s3validation = f's3://{bucket}/{prefix}/validation/'

train_data = sagemaker.session.s3_input(
    s3train,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.session.s3_input(
    s3validation,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, "validation": validation_data}

### Treinando o modelo

In [ ]:
ic.fit(inputs=data_channels, logs=True)

**Obs.:** Os hiperparâmetros irão influenciar no resultado obtido. Podemos otimizar o modelo efetuando um tuning de hiperparâmetro. Mas não iremos abordar isso por agora.

## Inferências

Um modelo treinado não faz nada por conta própria. Agora queremos usar o modelo para realizar inferências. Para este exemplo, isso significa prever a classe dos objetos. Você pode implantar o modelo criado usando o método de implantação do Estimator

In [ ]:
ic_classifier = ic.deploy(initial_instance_count = 1,
                                    instance_type = 'ml.m5.xlarge')

## Testando e avaliando nosso modelo

In [ ]:
from IPython.display import Image

test_image = 'images/morcego.jpg'
Image(test_image)

In [ ]:
with open(test_image, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
    
ic_classifier.content_type = 'application/x-image'
result = json.loads(ic_classifier.predict(payload))
# O resultado irá trazer a probabilidade de todas as classes
# Estamos utilizando a classe com maior probabilidade
index = np.argmax(result)
object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
print("Resultado: rótulo - " + object_categories[index] + ", probabilidade - " + str(result[index]))

In [ ]:
# Limpando os recursos - após finalizar o workshop lembre de deletar o endpoint de inferência
# ic_classifier.delete_endpoint() 